## Nested operators
This notebook explores how to build new operators via operator nesting. This is a design suggestion, i.e. the actual implementation has not been carried out yet. There are also design suggestions for simplying the detection of regions and applying an operator.

The key function that enables the construction of nested operators is `get_array`. This function could also be used for constructing the `banded` operation discussed in the [fills](fills.ipynb) notebook. Since this means that there are two ways of accomplish the same thing, one of them should be removed. My vote would be to keep the function `get_array` and remove the function `banded`. 

Another aspect that this notebook explores is the value access vs symbolic access. I'm wondering if not having `( )` for get value is a bad design idea because it will inevitably lead to problems when the indices are symbolic.

In [1]:
from openfd.alpha import GridFunction, Expression, Operator
from openfd.alpha import array

class SecondDerivative(Operator):
    
    def initialize(self):
        ## Construct an expression for a first derivative via operator nesting.
        D = Derivative('D') 
        D2 = Expression(D*D)      
        
        # Now, we get the data arrays from this second derivative.
        # The trick to building the `get_array` function would be to invoke the `__call__` method on the expression 
        # `D2` and get the indices and values from the expression itself. 
        # That is, the ability to perform value accesses on an expression is the main mechanism behind this function.
        # TODO: I think I have pretty good idea of how to accomplish the implementation of this function (not done yet).
        self._data[0] = get_array('d2l', D2, 0) # left boundary
        self._data[1] = get_array('d2i', D2, 1) # interior
        self._data[2] = get_array('d2r', D2, 2) # right boundary
        
        # For now we assign some other data to demonstrate usage
        self._data[0] = array.CArray('d2l', data = [[1.0, -2.0, 1.0]])
        self._data[1] = array.CArray('d2i', data = [[1.0, -2.0, 1.0]])
        self._data[2] = array.CArray('d2r', data = [1.0, -2.0, 1.0]) 
        
    def __getitem__(self, index):
        """
        TODO: This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, *index : x[index])
    
    def __call__(self, index):
        """
        TODO: This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, *index : x(*index))
    
    def __eval__(self, index, op):
        """
        The user will only need to implement this method. 
        By using the lambda `op`, it is only necessary to write the implementation once. 
        """   
        
        # The get region returns the region id for an index. The index can be either `int` or symbolic `Index`.
        # If `index` is an int, then it either maps to the region: 0 if `index > 0` (left), or region: 2 if `index < 0`. 
        # The `Index` class could also be fleshed out by having a set function that make it possible to store a value.
        region = get_region(index)      
        data = self.data[region]
        
        # Left or right boundary
        if region != 2: 
            return sum([op(self.args, j)*op(data, index, j) for j in range(data.shape[1])])
        # interior
        else:      
            return sum([op(self.args, j + index)*op(data, j) for j in range(data.shape[0])])
        
def get_region(index):
    """
    Determines what region an index maps to. 
    If the index is of type `Index` this information could be read from the `id` attribute as dicussed in another issue.
    If that is poor design descision, then it should be possible to determine the region from the index bounds.
    In case region is an `int`, then it can only be mapped to the left (region = `0`), or the right (region = `1`) regions.
    By convention, non-negative indices map to the left, and negative indices map to the right region.
    
    Arguments:
    
        index : The index to determine region affinity for.
    
    Returns:
    
        int : The region id.
    
    """
    if isinstance(index, int):
        if index < 0:
            return 2
        else:
            return 1
    else:
        return index.id

def get_array(label, expr, region, num_pts='auto', max_num_pts=100, language='C'):
    """
    Analyzes an expression that contains operators and packs their data into a
    data array. This function assumes that the operators are one dimensional.

    Arguments:
    
        label(`str`): The label to assign to the data array.
        expr : The expression to analyze. 
        region : An optional `int` that determines the region to extract data
                    from. Use `0` for left boundary, `1` for interior, or `2` for right boundary. Defaults
                    to `0`.
        num_pts : An optional `int` that gives the number of points to analyze. 
                    By default, this argument is set to `'auto'` which tries to
                    automatically detect the number of boundary points by stopping
                    when repetition occurs.
        max_num_pts : An optional `int` that sets the maximum number of grid
                          points to use when constructing the operator. Defaults to `100`.
        language : An optional `str` that specifies the language to use for the array. 
                       Defaults to `C`. See [array](array.ipynb) for details.

    Returns:
    
        Array : An instance of `Array` but for the target language. Defaults to `CArray`. 

    """
    return None
"""
TODO: Remove duplication. The derivative class has been copied from the notebook [operators-value-access](operators-value-access.ipynb)
"""
from openfd.alpha.array import CArray
class Derivative(Operator):
    
    def initialize(self):
        self._data[0] = CArray('dl', data=[-1.0, 1.0])
        self._data[1] = CArray('di', data=[-0.5, 0.5])
        self._data[2] = CArray('dr', data=[-1.0, 1.0])
        
    def __getitem__(self, index):
        """
        This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, index : x[index])
    
    def __call__(self, index):
        """
        This method will be moved to the base class.
        """
        return self.__eval__(index, lambda x, index : x(index))
    
    def __eval__(self, index, op):
        """
        The user will only need to implement this method. 
        By using the lambda `op`, it is only necessary to write the implementation once. 
        """
        left = self.data[0]
        interior = self.data[1]
        right = self.data[2]
        
        # TODO: Build some function that handles region detection for any input (int, or symbolic)
        if isinstance(index, int):
            if index == 0:
                return op(self.args, 0)*op(left, 0) + op(self.args, 1)*op(left, 1) 
            elif index == -1:
                return op(self.args, -1)*op(right, 0) + op(self.args, -2)*op(right, 1)
            else:
                return op(self.args, index-1)*op(interior, 0) + op(self.args, index+1)*op(interior, 1)  
        else:           
            if index.id == 0:
                return op(self.args, 0)*op(left, 0) + op(self.args, 1)*op(left, 1)
            elif index.id == 2:
                return op(self.args, -1)*op(right, 0) + op(self.args, -2)*op(right, 1)
            else:
                return op(self.args, index-1)*op(interior, 0) + op(self.args, index+1)*op(interior, 1)
"""
TODO: Remove duplication.
place-holder class that will be implemented if approved. Will also include bounds as discussed in issue #13
These three classes will be one class. The region id will be an attribute.
(shameless copy-and-paste from [fills](fills.ipynb))
"""
from sympy import Symbol
class LeftBoundaryIndex(Symbol):
    @property
    def id(self):
        return 0
    
class RightBoundaryIndex(Symbol):
    @property
    def id(self):
        return 2
    
class InteriorIndex(Symbol):
    @property
    def id(self):
        return 1



Now that we have built the second derivative we can go ahead and use it.

In [2]:
D2 = SecondDerivative('D2')
u = GridFunction('u', shape=(10,))
explicit = Expression(D2*u)

Let's perform some symbolic and value accesses.

In [3]:
explicit[0]

u[0]*d2i[0][0] + u[1]*d2i[0][1] + u[2]*d2i[0][2]

We can check that the second derivative behaves as expected by comparing it to a nested expression using first derivatives. The value accesses should give the same results. 

In [4]:
D = Derivative('D')
implicit = Expression(D*D*u)

The key difference between having an explicit form for the the second derivative is that the array values are packed into a single array. If we perform a symbolic access on the implicit second derivative, then we get two array multiplications for each term.

In [6]:
il = LeftBoundaryIndex('il')
explicit[il]

u[0]*d2l[il][0] + u[1]*d2l[il][1] + u[2]*d2l[il][2]

In [7]:
implicit[il]

(u[0]*di[0] + u[2]*di[1])*dl[1] + (u[0]*dl[0] + u[1]*dl[1])*dl[0]

In [15]:
explicit(0) # The values returned here are not correct at the moment.

1.0*u[0] - 2.0*u[1] + 1.0*u[2]

In [16]:
implicit(0)

-1.0*(-1.0*u[0] + 1.0*u[1]) + 1.0*(-0.5*u[0] + 0.5*u[2])

## Symbolic vs value access

In [14]:
# Does not work because we cannot pass a symbolic index to a get value-call. The value to get cannot be determined.
# explicit(il)

In [11]:
# Works here because the symbol is never passed to the array, which is sort of confusing.
implicit(il)

-1.0*(-1.0*u[0] + 1.0*u[1]) + 1.0*(-0.5*u[0] + 0.5*u[2])